In [2]:
import numpy as np
import pandas as pd
import dataframe_image as dfi

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

import folium
import json

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터프레임을 그림 파일로 저장
option = 1 # 1 : dfi / 2 : df2img
import df2img
def savedf(df, path):
    if option == 1:
        dfi.export(df, path)
    if option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)

# 속성 선택 (feature selection)

## 데이터 불러오기 및 병합

In [4]:
collected_data = pd.read_csv('../../data/data_d_final/2d201_1230_collected_data.csv')
collected_data.set_index("지역", inplace=True)
collected_data.head()

,위도,경도,쏘카존 수,녹지,주거,공업,상업,인구,대학교 수,아파트 수,지하철역 수,정류장 수,환승역 수,행복주택 수,대학생 수,2030인구
지역,,,,,,,,,,,,,,,,
수원시 장안구,37.303971,127.001743,18,0.272727,0.727273,0.000000,0.000000,271012,4,200,1,409,0,0,34441,82294.55980
수원시 권선구,37.257687,126.963532,16,0.625000,0.312500,0.062500,0.000000,367004,1,159,4,537,0,1,11033,111443.15610
수원시 팔달구,37.282570,127.011718,20,0.136364,0.409091,0.000000,0.454545,192225,0,65,4,185,2,1,0,58370.37385
수원시 영통구,37.259600,127.042341,34,0.142857,0.571429,0.285714,0.000000,360127,21,129,6,380,0,3,22911,109354.91020
용인시 처인구,37.234346,127.192965,29,0.823529,0.058824,0.000000,0.117647,259872,15,112,9,1234,0,1,13084,67270.69540


In [5]:
# collected_data.sort_values(by='지역',inplace=True)
# collected_data['dumb'] = np.arange(len(collected_data))
# collected_data

In [6]:
hackathon_data_original = pd.read_csv("../../data/20211022_수요예측_hackathon_data.csv", encoding='cp949')
hackathon_data_original.head()

,region1,region2,reservation_return_at,reservation_start_at,age_group,gender,car_model
0,울산광역시,남구,2019-09-29 21:25:40+00:00,2019-09-29 19:20:00+00:00,1,male,경형
1,울산광역시,남구,2019-07-13 13:11:21+00:00,2019-07-13 11:00:00+00:00,1,male,준중형
2,울산광역시,남구,2019-09-16 23:43:08+00:00,2019-09-16 19:10:00+00:00,1,male,준중형
3,울산광역시,남구,2019-08-09 06:17:41+00:00,2019-08-09 00:20:00+00:00,1,male,준중형
4,울산광역시,남구,2019-07-24 12:36:29+00:00,2019-07-24 09:00:00+00:00,2,male,경형


In [7]:
hackathon_data = pd.DataFrame(hackathon_data_original['region2'].value_counts())
hackathon_data.reset_index(inplace=True)
hackathon_data['index'].replace({ 
    '수원시\u3000팔달구':'수원시 팔달구', '수원시\u3000영통구':'수원시 영통구', '용인시\u3000기흥구':'용인시 기흥구',
    '고양시\u3000일산동구':'고양시 일산동구', '안양시\u3000동안구':'안양시 동안구', '고양시\u3000일산서구':'고양시 일산서구',
    '성남시\u3000수정구':'성남시 수정구', '수원시\u3000장안구':'수원시 장안구', '성남시\u3000중원구':'성남시 중원구',
    '안산시\u3000단원구':'안산시 단원구', '용인시\u3000수지구':'용인시 수지구', '고양시\u3000덕양구':'고양시 덕양구',
    '용인시\u3000처인구':'용인시 처인구', '안양시\u3000만안구':'안양시 만안구', '성남시\u3000분당구':'성남시 분당구',
    '수원시\u3000권선구':'수원시 권선구', '안산시\u3000상록구':'안산시 상록구',}, inplace=True)
hackathon_data.columns = ['지역', '이용 수']
hackathon_data.set_index('지역', inplace=True)
hackathon_data.head()

,이용 수
지역,
성남시 분당구,46782
화성시,35249
고양시 일산동구,31906
남양주시,26546
고양시 덕양구,26253


In [8]:
intersected_rows = set.intersection(set(collected_data.index), set(hackathon_data.index))
intersected_rows = list(intersected_rows)
print(len(intersected_rows),'개 지역 \n',intersected_rows)

16 개 지역 
 ['고양시 일산서구', '성남시 중원구', '남양주시', '구리시', '성남시 분당구', '양평군', '성남시 수정구', '고양시 덕양구', '안양시 만안구', '고양시 일산동구', '하남시', '화성시', '광명시', '김포시', '의정부시', '안양시 동안구']


In [9]:
collected_data_intersected = collected_data[collected_data.index.isin(intersected_rows)]
collected_data_intersected.sort_index(ascending=True, inplace=True)
hackathon_data_intersected = hackathon_data[hackathon_data.index.isin(intersected_rows)]
hackathon_data_intersected.sort_index(ascending=True, inplace=True)
intersected = pd.concat([collected_data_intersected, hackathon_data_intersected],axis=1)
intersected['쏘카존 수 대비 이용 수'] = intersected['이용 수']/intersected['쏘카존 수']
intersected

,위도,경도,쏘카존 수,녹지,주거,공업,상업,인구,대학교 수,아파트 수,지하철역 수,정류장 수,환승역 수,행복주택 수,대학생 수,2030인구,이용 수,쏘카존 수 대비 이용 수
지역,,,,,,,,,,,,,,,,,,
고양시 덕양구,37.637431,126.823745,48,0.656250,0.343750,0.000000,0.000000,487874,6,274,9,1205,2,8,4210,131281.05900,26253,546.937500
고양시 일산동구,37.658622,126.771006,51,0.461538,0.461538,0.000000,0.076923,296590,1,133,6,454,0,0,13923,79808.82212,31906,625.607843
고양시 일산서구,37.677991,126.741105,29,0.500000,0.375000,0.000000,0.125000,290738,0,140,4,635,0,0,0,78234.11890,16052,553.517241
광명시,37.478618,126.856271,28,0.222222,0.666667,0.000000,0.111111,288182,0,89,3,445,0,0,0,76757.00000,22645,808.750000
구리시,37.594266,127.121253,19,0.500000,0.500000,0.000000,0.000000,188876,0,111,2,313,0,1,0,51033.00000,13011,684.789474
김포시,37.615350,126.707165,55,0.400000,0.533333,0.066667,0.000000,484194,3,229,9,1325,0,7,232,127499.00000,21500,390.909091
남양주시,37.635985,127.212283,57,0.526316,0.473684,0.000000,0.000000,737366,2,385,13,1803,0,6,8100,176762.00000,26546,465.719298
성남시 분당구,37.382819,127.110546,58,0.500000,0.500000,0.000000,0.000000,482026,1,231,12,694,3,0,0,137576.86410,46782,806.586207
성남시 수정구,37.450396,127.137254,18,0.631579,0.368421,0.000000,0.000000,232125,11,91,9,436,2,5,20021,66251.67434,18773,1042.944444


## 상관분석 (pearson correlation analysis)

In [10]:
pearson = intersected.corr()[['이용 수', '쏘카존 수 대비 이용 수']]
pearson.drop(index = ['이용 수', '쏘카존 수 대비 이용 수'], inplace=True)
pearson

,이용 수,쏘카존 수 대비 이용 수
위도,-0.207992,-0.496508
경도,-0.393671,0.128923
쏘카존 수,0.806230,-0.537069
녹지,-0.369392,-0.148646
주거,0.381949,0.060353
공업,-0.004037,0.237283
상업,-0.011755,0.166735
인구,0.649762,-0.390717
대학교 수,0.153327,0.012843
아파트 수,0.623385,-0.388349


In [11]:
savedf(pearson, "results/1b_201_0112_pearson.png")

In [12]:
pearson[pearson.abs() > 0.15]

,이용 수,쏘카존 수 대비 이용 수
위도,-0.207992,-0.496508
경도,-0.393671,NaN
쏘카존 수,0.806230,-0.537069
녹지,-0.369392,NaN
주거,0.381949,NaN
공업,NaN,0.237283
상업,NaN,0.166735
인구,0.649762,-0.390717
대학교 수,0.153327,NaN
아파트 수,0.623385,-0.388349


In [13]:
# sns.pairplot(intersected)

## 회귀분석 (linear regression analysis)

### sklearn

In [14]:
intersected.columns

Index(['위도', '경도', '쏘카존 수', '녹지', '주거', '공업', '상업', '인구', '대학교 수', '아파트 수',
       '지하철역 수', '정류장 수', '환승역 수', '행복주택 수', '대학생 수', '2030인구', '이용 수',
       '쏘카존 수 대비 이용 수'],
      dtype='object')

In [15]:
independents = intersected.copy()
independents = independents.drop(["이용 수", "쏘카존 수 대비 이용 수"], axis=1)
dependent = intersected["이용 수"]
independents.shape, dependent.shape

((16, 16), (16,))

In [16]:
# R squared의 값은 1에 가깝게 나온다. 독립변수들(feature들)을 하나나 두 개라도 줄이면 R squared의 값은 0.99xxx로 나타난다.
reg = LinearRegression().fit(independents, dependent)
print("R squared :",reg.score(independents, dependent))

R squared : 1.0


In [17]:
pd.DataFrame({'features':independents.columns, "coefficients":reg.coef_})

,features,coefficients
0,위도,-58292.804651
1,경도,16355.045846
2,쏘카존 수,826.879031
3,녹지,-34221.769801
4,주거,-10542.810325
5,공업,-50648.668942
6,상업,95413.249070
7,인구,-0.169465
8,대학교 수,-232.376012
9,아파트 수,190.707798


In [18]:
# 이해할 수 없는 결과가 나왔다. 위도, 경도, 녹지, 주거, 공업, 상업의 coefficient가 굉장히 크게 나왔다.
# 이것은 직관과도 일치하지 않을 뿐더러 상관관계의 결과와도 일치하지 않는다.
# sklearn의 공식문서를 보면 스케일링을 해주는 것이 필요할 수도 있다고 했다.

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
independents_scaled = scaler.fit_transform(independents.to_numpy())
independents_scaled = pd.DataFrame(independents_scaled, columns=independents.columns, index=independents.index)
independents_scaled.head()

,위도,경도,쏘카존 수,녹지,주거,공업,상업,인구,대학교 수,아파트 수,지하철역 수,정류장 수,환승역 수,행복주택 수,대학생 수,2030인구
지역,,,,,,,,,,,,,,,,
고양시 덕양구,0.813094,0.151001,0.811321,0.558036,0.458333,0.0,0.000000,0.465198,0.222222,0.673352,0.40,0.363844,0.666667,0.470588,0.210279,0.460164
고양시 일산동구,0.852445,0.082691,0.867925,0.307692,0.615385,0.0,0.615385,0.221678,0.037037,0.269341,0.25,0.077422,0.000000,0.000000,0.695420,0.244822
고양시 일산서구,0.888413,0.043961,0.452830,0.357143,0.500000,0.0,1.000000,0.214228,0.000000,0.289398,0.15,0.146453,0.000000,0.000000,0.000000,0.238234
광명시,0.518186,0.193130,0.433962,0.000000,0.888889,0.0,0.888889,0.210974,0.000000,0.143266,0.10,0.073989,0.000000,0.000000,0.000000,0.232054
구리시,0.732939,0.536347,0.264151,0.357143,0.666667,0.0,0.000000,0.084549,0.000000,0.206304,0.05,0.023646,0.000000,0.058824,0.000000,0.124434


In [20]:
reg = LinearRegression().fit(independents_scaled, dependent)
print("R squared :",reg.score(independents_scaled, dependent))

R squared : 1.0


In [21]:
result = pd.DataFrame({'features':independents.columns, "coefficients":reg.coef_})
result

,features,coefficients
0,위도,-31391.724573
1,경도,12626.960576
2,쏘카존 수,43824.588622
3,녹지,-9852.056115
4,주거,8259.022639
5,공업,-3232.647603
6,상업,14621.011197
7,인구,-133113.553726
8,대학교 수,-6274.152319
9,아파트 수,66557.021504


In [33]:
result['abs coef.'] = abs(result['coefficients'])
result.sort_values(by='abs coef.', ascending=False, inplace=True)
result.drop(columns = ['abs coef.'], inplace=True)
result

,features,coefficients
7,인구,-133113.553726
9,아파트 수,66557.021504
2,쏘카존 수,43824.588622
0,위도,-31391.724573
11,정류장 수,25174.597670
6,상업,14621.011197
15,2030인구,13166.869651
1,경도,12626.960576
3,녹지,-9852.056115
12,환승역 수,8261.080976


In [23]:
# 아까보다는 나아졌지만, 여전히 이상한 결과이기는 하다.
# 위의 결과를 통해서 보면 '이용 수'에 많은 영향을 미치는 요인은 '인구', '아파트 수', '쏘캬존 수'. '위도', '정류장 수', '상업' 순이다.
# 이 순서 자체는 이상할 게 없을 수 있으나, '인구'의 계수는 음수이다. 다시 말해, 인구가 적을 수록 이용수가 많다는 말이므로, 이 결과는 이상하다.

In [36]:
import statsmodels.api as sm
#define response variable
y = dependent
#define predictor variables
x = independents
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   이용 수   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 26 Jan 2023   Prob (F-statistic):                nan
Time:                        17:31:43   Log-Likelihood:                 173.51
No. Observations:                  16   AIC:                            -315.0
Df Residuals:                       0   BIC:                            -302.6
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.095e+05        inf          0        n

In [46]:
result = pd.DataFrame(model.params, columns=['coefficients'])
result['abs coef.'] = abs(result['coefficients'])
result.sort_values(by='abs coef.', ascending=False, inplace=True)
result.drop(columns = ['abs coef.'], inplace=True)
result

,coefficients
상업,122793.570535
const,109521.286547
위도,-58292.804586
공업,-23268.347341
주거,16837.511283
경도,16355.045829
녹지,-6841.448163
환승역 수,2753.693654
쏘카존 수,826.879030
대학교 수,-232.376012


In [ ]:
# 여전히 이상한 결과가 나왔다. min-max scaling을 하지 않은 결과일까?
# min-max scaling을 적용해서 다시 해보았다.

In [47]:
import statsmodels.api as sm
#define response variable
y = dependent
#define predictor variables
x = independents_scaled
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   이용 수   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 26 Jan 2023   Prob (F-statistic):                nan
Time:                        17:34:51   Log-Likelihood:                 355.61
No. Observations:                  16   AIC:                            -679.2
Df Residuals:                       0   BIC:                            -666.9
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7145.1859        inf          0        n

In [48]:
result = pd.DataFrame(model.params, columns=['coefficients'])
result['abs coef.'] = abs(result['coefficients'])
result.sort_values(by='abs coef.', ascending=False, inplace=True)
result.drop(columns = ['abs coef.'], inplace=True)
result

,coefficients
인구,-133113.553726
아파트 수,66557.021504
쏘카존 수,43824.588622
위도,-31391.724573
정류장 수,25174.597670
상업,15202.126846
2030인구,13166.869651
경도,12626.960576
주거,11745.716536
환승역 수,8261.080976


In [ ]:
# 아까보다 나은 결과인 것으로 보인다.
# 그리고 이것은 sklearn으로 얻은 결과와도 일치한다.